<font size=4 face='黑体'>使用CNN进行MNIST数据集分类

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('F:\data\mnist_data\MNIST_data',one_hot=True)

H:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting F:\data\mnist_data\MNIST_data\train-images-idx3-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\train-labels-idx1-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\t10k-labels-idx1-ubyte.gz


In [6]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def conv2d(x,W):
    #x为输入数据，形状为[N,H,W,C]；W为滤波器，形状为[H,W,C,N]
    #strides[0]和stridess[3]只能等于1，strides[1,2]分别表示x，y方向上的步幅
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

x_image = tf.reshape(x,[-1,28,28,1])

W_conv1 = weight_variable([4,4,1,16])  #4 * 4 的采样窗口，16个滤波器一共生成16张特征图
b_conv1 = bias_variable([16])

#28*28*1 的图片卷积之后变为 28*28*16
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
#池化后变成 14*14*16
h_pool1 = max_pool_2x2(h_conv1)

#第二次卷积之后变为 14*14*16
W_conv2 = weight_variable([4,4,16,16])
b_conv2 = bias_variable([16])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
#第二次池化后变为 7*7*16
h_pool2 = max_pool_2x2(h_conv2)

#第一个全连接层 
W_fc1 = weight_variable([7*7*16,500])
b_fc1 = bias_variable([500])
#7*7*16的图像输出变为1维向量进行输入
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*16])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#第二个全连接层
W_fc2  = weight_variable([500,10])
b_fc2 =  bias_variable([10])
a2 = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
prediction = tf.nn.softmax(a2)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=a2))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:0.7})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,
                                          keep_prob:1.0})
        print('Iter'+str(epoch)+' Testing Accuracy:'+str(acc))


Iter0 Testing Accuracy:0.9796
Iter1 Testing Accuracy:0.9834
Iter2 Testing Accuracy:0.9895
Iter3 Testing Accuracy:0.9885
Iter4 Testing Accuracy:0.9901
Iter5 Testing Accuracy:0.9917
Iter6 Testing Accuracy:0.9909
Iter7 Testing Accuracy:0.99
Iter8 Testing Accuracy:0.9907
Iter9 Testing Accuracy:0.9923
Iter10 Testing Accuracy:0.992
Iter11 Testing Accuracy:0.9918
Iter12 Testing Accuracy:0.9914
Iter13 Testing Accuracy:0.9922
Iter14 Testing Accuracy:0.9915
Iter15 Testing Accuracy:0.9905
Iter16 Testing Accuracy:0.9914
Iter17 Testing Accuracy:0.9929
Iter18 Testing Accuracy:0.9912
Iter19 Testing Accuracy:0.993
Iter20 Testing Accuracy:0.9921


<font size=4 face='黑体'>构建网络结构,并可视化数据

In [2]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size
max_step = 1001
k = 0.8

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape=shape),name='b')

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME',name='conv2d')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Max_pool')

#记录变量的函数
def variable_summary(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
#结构化卷积层的函数
def conv_layer(input_tensor,weight_shape,layer_name):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable(weight_shape)
            variable_summary(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([weight_shape[-1]])
            variable_summary(biases)
        with tf.name_scope('conv_comput'):
            preactivate = conv2d(input_tensor,weights) + biases
        with tf.name_scope('activate'):
            activations = tf.nn.relu(preactivate)
        return activations
    
#结构化全连接层的函数
def linear_layer(input_tensor,input_dim,output_dim,layer_name):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim,output_dim])
            variable_summary(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summary(biases)
        with tf.name_scope('linear_comput'):
            preactivate = tf.matmul(input_tensor,weights) + biases
        return preactivate
    
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x_input')
    with tf.name_scope('input_reshape'):
        x_image = tf.reshape(x,[-1,28,28,1],name='x_image')
        #tf.summary.image('input',x_image,10)
    y = tf.placeholder(tf.float32,[None,10],name='y_input')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    
#第一次卷积 28*28*1 -> 28*28*16
conv_layer1 = conv_layer(x_image,[4,4,1,16],'conv_layer1')
#第一次池化 28*28*16 -> 14*14*16
with tf.name_scope('Max_pool1'):
    h_pool1 = max_pool_2x2(conv_layer1)
    
#第二次卷积 14*14*16 -> 14*14*32
conv_layer2 = conv_layer(h_pool1,[3,3,16,32],'conv_layer2')
#第二次池化 14*14*32 -> 7*7*32
with tf.name_scope('Max_pool2'):
    h_pool2 = max_pool_2x2(conv_layer2)

with tf.name_scope('flatten'):
    flatten = tf.reshape(h_pool2,[-1,7*7*32])

#第一个全连接层
fc1 = linear_layer(flatten,7*7*32,500,'FC1')

with tf.name_scope('activate'):
    activations = tf.nn.relu(fc1)
        
with tf.name_scope('dropout'):
    fc1_drop = tf.nn.dropout(activations,keep_prob)
    
#第二个全连接层
fc2 = linear_layer(fc1_drop,500,10,'FC2')

with  tf.name_scope('activate'):
    prediction = tf.nn.softmax(fc2)
    
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=fc2))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
    
with tf.name_scope('Accuracy'):
    correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    tf.summary.scalar('accuracy',accuracy)

merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(r'F:\data\mnist_data\logs',sess.graph)
    
    for i in range(max_step):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:k})
        
        if i%10 == 0:
            summary,acc = sess.run([merged,accuracy],feed_dict={x:mnist.test.images,y:mnist.test.labels,
                                          keep_prob:1.})
            writer.add_summary(summary,i)
            print('Step'+str(i)+' Testing Accuracy:'+str(acc))
            
    writer.close()

Step0 Testing Accuracy:0.1314
Step10 Testing Accuracy:0.6259
Step20 Testing Accuracy:0.7745
Step30 Testing Accuracy:0.8511
Step40 Testing Accuracy:0.8828
Step50 Testing Accuracy:0.9059
Step60 Testing Accuracy:0.9172
Step70 Testing Accuracy:0.9241
Step80 Testing Accuracy:0.9337
Step90 Testing Accuracy:0.9344
Step100 Testing Accuracy:0.9421
Step110 Testing Accuracy:0.948
Step120 Testing Accuracy:0.9544
Step130 Testing Accuracy:0.9521
Step140 Testing Accuracy:0.953
Step150 Testing Accuracy:0.9535
Step160 Testing Accuracy:0.9564
Step170 Testing Accuracy:0.9593
Step180 Testing Accuracy:0.9639
Step190 Testing Accuracy:0.9644
Step200 Testing Accuracy:0.9625
Step210 Testing Accuracy:0.9616
Step220 Testing Accuracy:0.969
Step230 Testing Accuracy:0.966
Step240 Testing Accuracy:0.9708
Step250 Testing Accuracy:0.9746
Step260 Testing Accuracy:0.9732
Step270 Testing Accuracy:0.9745
Step280 Testing Accuracy:0.9751
Step290 Testing Accuracy:0.9763
Step300 Testing Accuracy:0.9748
Step310 Testing Accurac